In [1]:
import csv

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


La manera preferida de leer un archivo CSV es utilizando el método `DictReader()`, el cual crea una un diccionario por cada línea que lee, con los nombres de columna como clave, y los contenidos del registro como valor.

In [2]:
import os
data_pkg_path = 'data'
filename = 'worldcities.csv'
path = os.path.join(data_pkg_path, filename)

In [ ]:
f = open(path, 'r')
csv_reader = csv.DictReader(f, delimiter=',', quotechar='"')
print(csv_reader)
f.close()

## Utilizando la función `enumerate()`

Cuando se itera sobre un objeto, muchas veces necesitamos un contador, para ir llevando la cuenta del índice de cada elemento.

Una forma de conseguir esto es llevando manualmente la cuenta de cada elemento, pero Python permite hacerlo automáticamente mediante la función `enumerate()`.

In [1]:
cities = ['San Francisco', 'Los Angeles', 'New York', 'Atlanta']
for x in enumerate(cities):
    print(x)

(0, 'San Francisco')
(1, 'Los Angeles')
(2, 'New York')
(3, 'Atlanta')


Podemos usar `enumerate()` en cualquier objeto iterable y obtener una serie de tuplas con el índice y el valor en dicha iteración.

### Ejemplo
Veamos cómo leer los 5 primeros elementos de un diccionario

In [ ]:
f = open(path, 'r', encoding='utf-8')
csv_reader = csv.DictReader(f, delimiter=',', quotechar='"')
for index, row in enumerate(csv_reader):
    print(row)
    if index == 4:
        break
f.close()

The code for file handling requires we open a file, do something with the file object and then close the file. That is tedious and it is possible that you may forget to call `close()` on the file. If the code for processing encounters an error the file is not closed property, it may result in bugs - especially when writing files.

The preferred way to work with file objects is using the `with` statement. It results in simpler and cleaer code - which also ensures file objects are closed properly in case of errors.

As you see below, we open the file and use the file object `f` in a `with` statement. Python takes care of closing the file when the execution of code within the statement is complete.

In [ ]:
with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

## Filtrando registros

Podemos utilizar sentencias condicionales cuando iteramos sobre los registros de un CSV para seleccionar y procesar aquellos registros que caen bajo determinado criterio. 

### Ejemplo

Contemos cuántas ciudades de un determinado país están presentes en el archivo.

In [3]:
country = 'Argentina'
num_cities = 0

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

    for row in csv_reader:
        if row['country'] == country:
            num_cities += 1
            
print(num_cities)

NameError: name 'path' is not defined

## Calculando distancia

Apliquemos lo que hemos aprendido hasta ahora para resolver un problema. Queremos leer el archivo `worldcities.csv`, encontrar todas las ciudades que pertenecen a un país dado, y luego calcular las distancias de una ciudad que elijamos a dichas ciudades. Los resultados deben ser escritos a un nuevo archivo CSV.

First we find the coordinates of the out selected `home_city` from the file. Replace the `home_city` below with your hometown or a large city within your country. Note that we are using the `city_ascii` field for city name comparison, so make sure the `home_city` variable contains the ASCII version of the city name.

In [ ]:
home_city = 'Bengaluru'

home_city_coordinates = ()

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        if row['city_ascii'] == home_city:
            lat = row['lat']
            lng = row['lng']
            home_city_coordinates = (lat, lng)
            break
        
print(home_city_coordinates)

Ahora podemos recorrer el archivo, encontrar una ciudad en el país elejido y llamar a la función `geopy.distance.geodesic()` para calcular la distancia.

In [4]:
from geopy import distance

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        if (row['country'] == home_country and
            row['city_ascii'] != home_city):
            city_coordinates = (row['lat'], row['lng'])
            city_distance = distance.geodesic(
                city_coordinates, home_city_coordinates).km
            print(row['city_ascii'], city_distance)


        

ModuleNotFoundError: No module named 'geopy'

## Escribiendo archivos

En vez de imprimir los resultados en pantalla, vamos a escribirlos en un nuevo archivo. De manera similar a `csv. DictReader()`, existe la función `csv. DictWriter()` para escribir archivos. Primero creamos un objeto `csv_writer` y luego escribimos en él usando el método `writerow()`.

Primero creamos una carpeta `output` para guardar los resultados. Chequeamos si existe, en caso negativo la creamos.

In [ ]:
output_dir = 'output'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
output_filename = 'cities_distance.csv'
output_path = os.path.join(output_dir, output_filename)

distances = []

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        country = row['country']
        city = row['city_ascii']
        if country == home_country and city != home_city:
            city_coordinates = (row['lat'], row['lng'])
            city_distance = distance.geodesic(city_coordinates, home_city_coordinates).km
            distances.append((city, city_distance))

with open(output_path, mode='w', newline='', encoding='utf-8') as output_file:
    fieldnames = ['city', 'distance_from_home']
    csv_writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for city, distance in distances:
        csv_writer.writerow(
            {'city': city,
             'distance_from_home': distance}
            )

## Ejercicios

- (Fácil) Reemplazar el valor de las variables de la ciudad de origen y el país y crear un nuevo archivo csv conteniendo las distancias de esa ciudad de origen a las ciudades de ése país.
- (No tan fácil) Hacer un código equivalente pero más corto. Sugerencia: usar [listas por comprensión](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) o bien `map()` y `filter()` y la función `csv.writerows()`

----